In [11]:
import sys
import os.path as osp
import pprint
import networkx as nx
from tqdm import tqdm
from networkx.algorithms import bipartite
import numpy as np
import pandas as pd
import re
import json
import torch

SRC_SUBDIR = '/app/src/'
SRC_SUBDIR = osp.abspath(SRC_SUBDIR)
if SRC_SUBDIR not in sys.path:
    print(f'Adding source directory to the sys.path: {SRC_SUBDIR!r}')
    sys.path.insert(1, SRC_SUBDIR)
pprint.pprint(sys.path)

from util import *

['/app/notebooks',
 '/app/src',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '',
 '/usr/local/lib/python3.8/dist-packages',
 '/usr/lib/python3/dist-packages']


In [12]:
df1 = load_binary('/app/data/df_with_term.pkl')
len(df1)

44564

In [13]:
df1['terms'] = df1['term_all']

# terms内の”例”が含まれる単語を削除
df1['terms'] = df1['terms'].apply(lambda x: [t for t in x if '例' not in t])
# termsをlistからstrに変換
df1['terms'] = df1['terms'].apply(lambda x: ' '.join(x))


In [14]:
# df1 = load_binary('/app/data/patent_df2021.pkl')
df2 = load_binary('/app/data/patent_df2023.pkl')

df = pd.concat([df1, df2], axis=0)
df = df.reset_index(drop=True)

In [15]:
df

,id,company,IPC,f_term,term_all,year,terms,出願番号,head_IPC,sentence
0,0.0,佐藤工業株式会社,B65D,3E060AA03|3E060AB05|3E060BC02|3E060CB04|3E060C...,一実施例|一対|上縁|仕切り壁|仕切り板|仕切板|加圧|圧縮強度|実施例|技術分野|箱本体|連設,2001,一 実 施 | 一 対 | 上 縁 | 仕 切 り 壁 | 仕 切 り 板 | 仕 切 板 ...,NaN,NaN,NaN
1,1.0,寄神建設株式会社|東亜建設工業株式会社|社団法人日本海上起重技術協会,B66C,3F004CC01|3F004CD08,下げワイヤ|回動|実施形態|技術分野|玉掛けワイヤ|遠隔操作,2001,下 げ ワ イ ヤ | 回 動 | 実 施 形 態 | 技 術 分 野 | 玉 掛 け ワ ...,NaN,NaN,NaN
2,2.0,株式会社大林組,E01D,2D059AA03|2D059CC04,コンクリート打設|コンクリート打設工|コンクリート構造|一実施例|作業用足場|作業足場|係止...,2001,コ ン ク リ ー ト 打 設 | コ ン ク リ ー ト 打 設 工 | コ ン ク リ ...,NaN,NaN,NaN
3,3.0,株式会社大林組,E02B|B65G,3F040BA01|3F040CA04|3F040EA03,ケーブルクレーン|コンクリートダム|コンクリートバケット|コンクリートプラント|コンクリート...,2001,ケ ー ブ ル ク レ ー ン | コ ン ク リ ー ト ダ ム | コ ン ク リ ー ...,NaN,NaN,NaN
4,4.0,株式会社大林組,E02B,NaN,ケーブルクレーン|コンクリートダム|コンクリートバケット|コンクリートプラント|コンクリート...,2001,ケ ー ブ ル ク レ ー ン | コ ン ク リ ー ト ダ ム | コ ン ク リ ー ...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
49355,NaN,株式会社竹中工務店,NaN,2D147AB00|2D147AB04|2E172AA05|2E172DA01|2E172D...,NaN,2022,掘削領域,特願2022-110399,E02D,【発明の詳細な説明】【技術分野】【０００１】本発明は、構造物の施工方法に関する。【背景技術】...
49356,NaN,株式会社竹中工務店,NaN,2D147AB00|2D147AB04,NaN,2022,掘削領域|施工工程,特願2022-110398,E02D,【発明の詳細な説明】【技術分野】【０００１】本発明は、構造物の施工方法に関する。【背景技術】...
49357,NaN,株式会社竹中工務店,NaN,2D147AB04,NaN,2022,山留め壁|掘削領域,特願2021-208648,E02D,【発明の詳細な説明】【技術分野】【０００１】本発明は、逆打ち工法に関する。【背景技術】【００...
49358,NaN,株式会社フジタ,NaN,2D155CA03|2D155DA08|2D155KA00|2D155KC01|2D155K...,NaN,2022,レベルセンサー|型枠本体,特願2021-173966,E21D,【発明の詳細な説明】【技術分野】【０００１】本発明は覆工コンクリート打設装置に用いるセンサー...


In [16]:
company_feat_df = df.set_index('出願番号')['company'].str.get_dummies('|')

In [17]:
company_feat_df

,▲吉▼岡建設株式会社,▲容▼鼎科技股▲分▼有限公司,▲蓮▼見 孝志,▲高▼井産業株式会社,▲高▼松建設株式会社,あいおい損害保険株式会社,あおみ建設株式会社,あき電器株式会社,いであ株式会社,つくばソフトウェアエンジニアリング株式会社,...,Ｔａｋｕｍｉ Ｖｉｓｉｏｎ株式会社,ＴｅｃｈＳｈａｒｅ株式会社,Ｔｏｐｏｌｏｇｉｃａｌ Ａｒｔｓ株式会社,ＵＢＥ三菱セメント株式会社,ＷＩＬＬコンサルタント株式会社,ＷＰＣコーポレーション株式会社,ＹＫＫ ＡＰ株式会社,ＹＫＫ株式会社,ｄｏｍｉ環境株式会社,ｍｔｅｓ Ｎｅｕｒａｌ Ｎｅｔｗｏｒｋｓ株式会社
出願番号,,,,,,,,,,,,,,,,,,,,,
NaN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NaN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NaN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NaN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NaN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
特願2022-110399,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
特願2022-110398,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
特願2021-208648,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
len(company_feat_df)

49360

In [19]:
term_feat_df = df.set_index('出願番号')['terms'].str.get_dummies('|')

In [20]:
term_feat_df

,か ん ぬ き 部 材,く さ び 部 材,ず れ 止 め,ず れ 止 め 部 材,せ ん 断 キ ー,せ ん 断 パ ネ ル,せ ん 断 ピ ン,せ ん 断 プ レ ー ト,せ ん 断 伝 達 部 材,せ ん 断 力 伝 達,...,ｐｃａ造,ｐｃａ部材,ｐｃｔ曲線,ｑｒコード,ｒｃスラブ,ｒｃ部材,ｒｆｉｄタグ,ｒｆｉｄリーダ,ｒｇｂ画像,ｓｒｃ造
出願番号,,,,,,,,,,,,,,,,,,,,,
NaN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NaN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NaN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NaN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NaN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
特願2022-110399,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
特願2022-110398,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
特願2021-208648,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
len(term_feat_df)

49360

In [22]:
def make_node(df:pd.DataFrame, column_name:str):
    node_list:list = []
    table = str.maketrans({
    '\u3000': '',
    ' ': '',
    '\t': ''
    })
    exeption_words = r'.+例'
    df[column_name] = df[column_name].astype(str)
    for targets in df[column_name].values:
        target = targets.split('|')
        for t in target:
            t = t.translate(table)
            if t not in node_list:
                if re.search(exeption_words, t):
                    continue
                else:
                    node_list.append(t)
    
    return node_list

In [23]:
company_node = make_node(df, 'company')
save_binary(company_node, '/app/data/new2/company_node_new.pkl')

'Done'

In [24]:
len(company_node)

4738

In [25]:
company_node.index('株式会社熊谷組')

28

In [26]:
term_node = make_node(df, 'terms')
save_binary(term_node, '/app/data/new2/term_node_new.pkl')

'Done'

In [27]:
def build_bipartite_graph(df, company_node, term_node):
    G = nx.Graph()
    G.add_nodes_from(company_node, bipartite='company')
    G.add_nodes_from(term_node, bipartite='term')

    table = str.maketrans({
    '\u3000': '',
    ' ': '',
    '\t': ''
    })
    exeption_words = r'.+例'
    for row in tqdm(df.itertuples()):
        companys = row.company.split('|')
        _ = row.terms.split('|')
        term = [t for t in _ if not re.search(exeption_words, t)]
        
        for c in companys:
            c = c.translate(table)
            for t in term:
                t = t.translate(table)
                G.add_edge(c, t)
    return G

In [28]:
H = build_bipartite_graph(df, company_node, term_node)
save_binary(H, '/app/data/new2/bipartite_graph_new.pkl')

49360it [00:02, 20164.57it/s]


'Done'

In [29]:
bi_adj:dict = bipartite.biadjacency_matrix(H, row_order=company_node, column_order=term_node)
save_binary(bi_adj, '/app/data/new2/bipartite_adj_new.pkl')

'Done'

In [30]:
print(bi_adj.todense().shape)

(4738, 9572)


In [31]:
def make_patent_any_relation(df, column_name, node_list):
    table = str.maketrans({
    '\u3000': '',
    ' ': '',
    '\t': ''
    })

    data = np.array([0]*len(df)*len(node_list)).reshape(len(df), len(node_list))
    
    for i in tqdm(range(len(df))):
        targets = df[column_name].values[i].split('|')
        for t in targets:
            t = t.translate(table)
            if t in node_list:
                data[i, node_list.index(t)] = 1
    
    return data

In [32]:
patent_company = make_patent_any_relation(df, 'company', company_node)
save_binary(patent_company, '/app/data/new2/patent_company_new.pkl')

patent_term = make_patent_any_relation(df, 'terms', term_node)
save_binary(patent_term, '/app/data/new2/patent_term_new.pkl')

100%|██████████| 49360/49360 [00:00<00:00, 77370.09it/s] 


100%|██████████| 49360/49360 [00:31<00:00, 1584.06it/s]


'Done'

In [33]:
patent_term.shape

(49360, 9572)

In [34]:
# ndarrayをtorch.tensorに変換
patent_company = torch.from_numpy(patent_company).to(torch.float)

In [35]:
patent_term = torch.from_numpy(patent_term).to(torch.float)

In [36]:
cpc = torch.matmul(patent_company.T, patent_company)

In [37]:
cpt = torch.matmul(patent_company.T, patent_term)

In [38]:
cpt.shape

torch.Size([4738, 9572])

In [39]:
def fix_feat(feat):
    # 1以上の値を1にする
    feat[feat > 1] = 1
    return feat

cpc = fix_feat(cpc)
cpt = fix_feat(cpt)

In [40]:
cpc

tensor([[1., 0., 1.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 1., 0.],
        [0., 0., 0.,  ..., 1., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]], device='cpu')

In [41]:
cpt

tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cpu')

In [42]:
save_binary(cpc, '/app/data/new2/cpc_new.pkl')
save_binary(cpt, '/app/data/new2/cpt_new.pkl')

'Done'